In [2]:
import pandas as pd
import numpy as np

### Create breakfast from grocery list and nutrition value

#### Calories: 400-500, protein >20

In [3]:
# Upload recipes datasets

data_short=pd.read_csv('data_short.csv')

In [4]:
data_short.drop('Unnamed: 0', inplace=True, axis=1)

In [15]:
# Upload the list of non_key ingredients
spices=list(pd.read_csv('spices.csv')['spices'])
spices[:5]

['mustard', 'sumac', 'saffron', 'celery', 'oregano']

In [5]:
# Should be at home
home=['pepper', 'butter', 'onion', 'butter', 'olive oil', 'sugar', 'salt', 'water']

In [7]:
# Samples groccery list (to be uploaded by the user)

grocery=['apple', 'milk', 'hazelnut', 'cucumber', 'bread', 'tomato', 
         'cod fillets',  'rice', 'garlic', 'chicken', 'cheese', 'egg']
grocery.extend(home)

In [8]:
options=data_short[(data_short.meal=='breakfast')&(data_short.calories<550)&(data_short.calories>400)&(data_short.protein>20)]

In [9]:
non_ingredients=['meal','title','calories','protein','carbs','fat','sodium','steps']

In [16]:
non_ingredients.extend(spices)

In [17]:
ingredients=options.drop(non_ingredients, axis=1).columns

#### Filter the recipes based on the meal type and nutrition

In [22]:
options=data_short[(data_short.meal=='breakfast')&(data_short.calories<550)&(data_short.calories>400)&(data_short.protein>20)]

In [23]:
# products outside the groccery list
needed=[x for x in ingredients if x not in grocery]

In [24]:
# Keep only the recipes if the number of additional key ingredients doesnt exceed 3
sums=options[needed].sum(axis=1)
ind=[x for x in sums.index if sums.loc[x]<=3]
options=options.loc[ind]

In [25]:
# drop columns with unused ingredients
options.drop([x for x in ingredients if sum(options[x])==0], axis=1, inplace=True)

In [26]:
options.shape

(18, 124)

In [38]:
recommendation=options.loc[ind][['title', 'calories', 'protein', 'carbs', 'fat']]

recommendation['products to add']=pd.Series([[x for x in options.drop(non_ingredients, axis=1).columns if 
                                              (x not in grocery)&(options.loc[i][x]==1)] for i in ind] , index=ind)
recommendation['non-key elements']=pd.Series([[x for x in spices if 
                                              (x not in grocery)&(options.loc[i][x]==1)] for i in ind] , index=ind)

recommendation['# of products to add']=pd.Series([len([y for y in recommendation.loc[i]['products to add'] 
                                                       if y not in non_ingredients]) for i in ind], index=ind)

In [39]:
recommendation

,title,calories,protein,carbs,fat,products to add,non-key elements,# of products to add
751,Herbed Lamb Burgers,538.0,28.0,39.00,30.0,"[pita bread rounds, lamb, plain yogurt]","[parsley, cumin, cinnamon, mint]",3
2361,Pork Chops with Golden Onions and Wilted Tomat...,466.0,43.0,10.50,28.0,"[vinegar, pork loin chops, cherry tomatoes]",[],3
2817,Turkey Hash,461.0,21.0,33.50,27.0,"[gold potatoes, cooked turkey]",[],2
2870,Spinach and Cheese Strata,509.0,30.0,29.75,30.0,"[parmigiano, spinach, dijon mustard]",[nutmeg],3
3074,White Bean and Watercress Gratin,490.0,27.0,64.00,14.0,"[bread crumbs, bean, wine vinegar]",[watercress],3
3215,Braised Chicken in Sun-Dried Tomato Cream,418.0,55.0,15.75,15.0,"[whipping cream, dry wine]",[basil],2
4189,GruyÃ¨re Potato Gratin,445.0,24.0,42.25,20.0,[potato],[],1
5866,Chicken Wings with Curry-Yogurt Glaze,521.0,44.0,12.00,33.0,"[plain yogurt, curry powder]","[cumin, cilantro, ginger]",2
6801,Chicken Paprikas,463.0,59.0,11.75,20.0,"[chicken broth, sour cream]",[paprika],2
7201,Grilled Salmon with Lime Butter Sauce,456.0,35.0,2.50,34.0,"[lime, salmon]",[],2
